In [20]:
#----------- Part-1: Create Data Frame
#----------- Part-2: Create Feature Columns & Target Column
#----------- Part-3: Train Models
#----------- Part-4: Fit, Predict, Score

#----------- Other Questions

import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from datetime import datetime, date, timedelta
import warnings
warnings.filterwarnings('ignore')

with open('hourly_data.json','rb') as f:
    data=f.readlines()
hourly_data = json.loads(data[0])

In [37]:
# for key in hourly_data.items():
#     print key
# (u'2015-12-06', [45, 43, 40, 40, 40, 38, 39, 43, 61, 84, 120, 139, 143, 114, 84, 82, 73, 69, 63, 55, 51, 49, 47, 42])

In [2]:
#----------- Part-1: Create Data Frame
#
#----------- Year-Week#-Weekday-Date-Hour-#_of_parking
#


# Date-Hour-Count
# [(u'2014-06-30', 0, 43), (u'2014-06-30', 1, 41),..]
raw_data = []
for key,value in hourly_data.items():
    for i in range(24):
        raw_data.append((key,i,value[i]))

data_df = pd.DataFrame(raw_data, columns=['Date','Hour','#_of_parking'])


#        Year  Week#  Weekday
# 0      2014     27        1
# Decompose Date to Year, Week# & Weekday
pk_df = pd.DataFrame(map(lambda x: x.isocalendar(), pd.to_datetime(data_df['Date'])), columns=['Year','Week#','Weekday'])

  
# len(pk_df)=27768
pk_df['Date']=data_df['Date']
pk_df['Hour']=data_df['Hour']
pk_df['#_of_parking']=data_df['#_of_parking']
            

# len(use_data_df)=18264
# Based on 'Basic_Static.ipynb: part-7' Ignore data of year 2012 & 2013
use_data_df = pk_df[(pk_df['Date']>'2013-12-31')]

In [3]:
# print pd.to_datetime(data_df['Date'])[0].isocalendar()
# (2014, 27, 1)
use_data_df.head()

,Year,Week#,Weekday,Date,Hour,#_of_parking
0,2014,27,1,2014-06-30,0,43
1,2014,27,1,2014-06-30,1,41
2,2014,27,1,2014-06-30,2,38
3,2014,27,1,2014-06-30,3,39
4,2014,27,1,2014-06-30,4,58


In [8]:
#----------- Part-2: Create Feature Columns & Target Column
#
#----------- list of features: 53 week, 7 weekday, 24 hour, [3 awd, weekend, holiday, 8 hour period]
#
# ToDo: 
# Holiday list ?
# actural_work_day no 5 ?
# Hour period check ?

# feature_cols = list(['Week#','Weekday','Hour'])
# target_col = use_data_df.columns[-1]
# X_all = use_data_df[feature_cols]
# y_all = use_data_df[target_col]
# print y_all.head()

lst_feature = []

# Week#
for i in range(53):
    lst_feature += ['w#_'+str(i)]
    use_data_df['w#_'+str(i)] = map(lambda x:1 if x==i else 0 , use_data_df['Week#'])

    
# Weekday#
for i in range(1,8):
    lst_feature += ['d#_'+str(i)]
    use_data_df['d#_'+str(i)] = map(lambda x:1 if x==i else 0, use_data_df['Weekday'])

    
# Actural Work Day & Weekend & Holidays
lst_feature += ['awd','weekend','?holiday']

actural_work_day = [1,2,3,4,5]
weekend = [6,7]
holidays_lst = ['2014-01-01', '2015-01-01', '2016-01-01',
 '2014-05-26', '2015-05-25', '2016-05-30',
 '2014-07-04', '2015-07-03', '2015-07-04', '2016-07-04',
 '2014-09-01', '2015-09-07', '2015-09-05',
 '2014-11-27', '2014-11-28', '2015-11-26', '2015-11-27', '2016-11-24', '2016-11-25',
 '2014-12-25', '2015-12-25', '2015-12-25', '2015-12-26']

use_data_df['awd'] = map(lambda x:1 if x in actural_work_day else 0, use_data_df['Weekday'])
use_data_df['weekend'] = map(lambda x:1 if x in weekend else 0, use_data_df['Weekday'])
use_data_df['?holiday'] = map(lambda x:1 if x in holidays_lst else 0, use_data_df['Date'])


# Hour#
for i in range(24):
    lst_feature += ['h#_'+str(i)]
    use_data_df['h#_'+str(i)] = map(lambda x:1 if x==i else 0, use_data_df['Hour'])

    
# Hour_Period
lst_feature += ['h_0_5_21_24','h_5_7','h_7_19','h_19_21','h_0_6','h_6_12','h_12_18','h_18_24']

rh1 = range(0,5)+range(21,24)
use_data_df['h_0_5_21_24'] = map(lambda x:1 if x in rh1 else 0, use_data_df['Hour'])
use_data_df['h_5_7'] = map(lambda x:1 if x in range(5,7) else 0, use_data_df['Hour'])
use_data_df['h_7_19'] = map(lambda x:1 if x in range(7,19) else 0, use_data_df['Hour'])   
use_data_df['h_19_21'] = map(lambda x:1 if x in range(19,21) else 0, use_data_df['Hour'])

use_data_df['h_0_6'] = map(lambda x:1 if x in range(0,6) else 0, use_data_df['Hour'])
use_data_df['h_6_12'] = map(lambda x:1 if x in range(6,12) else 0, use_data_df['Hour'])
use_data_df['h_12_18'] = map(lambda x:1 if x in range(12,18) else 0, use_data_df['Hour'])   
use_data_df['h_18_24'] = map(lambda x:1 if x in range(18,24) else 0, use_data_df['Hour'])
    
# len(lst_feature)=95
use_data_df.head()

,Year,Week#,Weekday,Date,Hour,#_of_parking,w#_0,w#_1,w#_2,w#_3,...,h#_22,h#_23,h_0_5_21_24,h_5_7,h_7_19,h_19_21,h_0_6,h_6_12,h_12_18,h_18_24
0,2014,27,1,2014-06-30,0,43,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
1,2014,27,1,2014-06-30,1,41,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
2,2014,27,1,2014-06-30,2,38,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
3,2014,27,1,2014-06-30,3,39,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
4,2014,27,1,2014-06-30,4,58,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0


In [9]:
X_all = use_data_df[lst_feature]
y_all = use_data_df['#_of_parking']

In [13]:
#----------- Part-3: Train Models
#

from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.grid_search import GridSearchCV


In [22]:
(X_train, X_test, y_train, y_test) = train_test_split(X_all, y_all, test_size=0.1)
print len(X_train), len(X_test)

16437 1827


In [23]:
#----------- LinearRegression

reg_linear = LinearRegression()
reg_linear.fit(X_train, y_train)
y_lreg = reg_linear.predict(X_test)
reg_linear.score(X_all, y_all)

0.73279211441777647